<a href="https://colab.research.google.com/github/fenikowski/AccountOfficeWebsite/blob/master/Plantilla_Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Igor Fenikowski  <br>
Url: https://github.com/fenikowski/03MAIR--Algoritmos-de-Optimizacion--2020/blob/master/Seminario/Plantilla_Seminario_Algoritmos.ipynb<br>
Url Colab: https://colab.research.google.com/drive/1gD2QCehUReSlIX9qZIN4GmxLu7Kk25_d?usp=sharing<br>
Problema:
>2. Organizar los horarios de partidos de La Liga<br>

Descripción del problema:

>Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de
liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un
algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia. <br>
>Los horarios disponibles se conocen a priori y son los siguientes: <br>
> - Viernes: 20 <br>
> - Sábado: 12, 16, 18, 20 <br>
> - Domingo: 12, 16, 18, 20 <br>
> - Lunes: 20 <br>

>En primer lugar se clasifican los equipos en tres categorías según el numero de
seguidores( que tiene relación directa con la audiencia). Hay 3 equipos en la
categoría A, 11 equipos de categoría B y 6 equipos de categoría C.
Se conoce estadisticamente la audiencia que genera cada partido según los equipos que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos). Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce. Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes. Es posible la coincidencia de horarios pero en este caso la audiencia de cada partido se verá afectada y se estima que se reduce en porcentaje.


                                

In [1]:
# Importación de librerías necesarias

import pandas as pd
import copy
import random
import math
from itertools import combinations

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

In [2]:
# Si tratamos todos 20 equipos como equipos únicos, sin tener en cuenta las restricciones, obtenemos el siguiente calculo:
result = 0

# (maneras de sacar 10 partidos entre los 20 equipos dados)
for i in range(2,21,2):
  result += len(list(combinations(range(20),2)))

# (maneras de asignar las fechas a cada partido)
result *= 10 ** 10

print(result)

19000000000000


In [3]:
# Si tratamos todos 20 equipos como equipos únicos, teniendo en cuenta las restricciones (de al menos una partida el viernes y el lunes), obtenemos el siguiente calculo:
result = 0

# (maneras de sacar 10 partidos entre los 20 equipos dados)
for i in range(2,21,2):
  result += len(list(combinations(range(20),2)))

# (maneras de asignar las fechas a cada partido)
result *= 8 ** 10

print(result)

2040109465600


Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

In [4]:
# He elegido modelar los datos del problema en varios diccionarios y arrays para poder facilmente aplicar "penalizaciones" por coindencia o hora, 
# tanto como poder conseguir información clara sobre equipos disponibles y calendario creado.

# Equipos para los que hay que preparar calendario:
teams = {"A": 3, "B": 11, "C":6}

# Diccionario con audiencia que generan partidas de diferentes equipos los sábados a las 20:
category_converter = {"AA":2, "AB":1.3, "AC":1, "BB":0.9, "BC":0.75, "CC":0.47}

# Array con penelizaciones por organizar más que un partido a una hora especifica:
coincidences = [0, 1, 0.75, 0.55, 0.4, 0.3, 0.25, 0.22, 0.2, 0.2]

# Diccionario con horario de partidas:
# - "weight" se refiere a la ponderación asignada a cada hora, 
# - audiencia total inicializada,
# - array de partidas inicializado

scheldule = {
    "V20": { "games": [], "weight": 0.4, "audience": 0 },
    "S12": { "games": [], "weight": 0.55, "audience": 0 },
    "S16": { "games": [], "weight": 0.7, "audience": 0 },
    "S18": { "games": [], "weight": 0.8, "audience": 0 },
    "S20": { "games": [], "weight": 1, "audience": 0 },
    "D12": { "games": [], "weight": 0.45, "audience": 0 },
    "D16": { "games": [], "weight": 0.75, "audience": 0 },
    "D18": { "games": [], "weight": 0.5, "audience": 0 },
    "D20": { "games": [], "weight": 1, "audience": 0 },
    "L20": { "games": [], "weight": 0.4, "audience": 0 }
}

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

In [5]:
# La función objetivo es obtener calendario que proporcione el mayor número de audiencia en 10 partidas jugadas entre 20 equipos.
# Es un problema de maximización, ya que se trata de obtener el valor máximo de la audiencia.

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [6]:
# Ya que la complejidad de algoritmo por fuerza bruta es muy alta, el calculo para propuestos datos de entrada es muy largo
# y no es posible obtener ningún resultado en tiempo razonable. Por tanto para testearlo y demostrar que si que podría conseguir buenos resultados,
# me he permitido reducir el número de equipos de la entrada (solo en este caso) al 3 equipos de categoría A, 3 de categoría B y 2 de categoría C.

In [7]:
teams = {"A": 3, "B": 3, "C":2}

In [8]:
# FUNCIONES AUXILIARES
# funcion para calcular partidas posibles de jugar

def count_available_games(teams):
  return {
      "AA": { "games": teams["A"] // 2, "audience": 2 },
      "BB": { "games": teams["B"] // 2, "audience": 0.9 },
      "CC": { "games": teams["C"] // 2, "audience": 0.47 },
      "AB": { "games": min(teams["A"], teams["B"]), "audience": 1.3 },
      "AC": { "games": min(teams["A"], teams["C"]), "audience": 1 },
      "BC": { "games": min(teams["B"], teams["C"]), "audience": 0.75 }
  }


# funcion para obtener partidas de diferentes categorías que se puede generar

def options_avaible(available_games):
  total = []
  for key, value in available_games.items():
    if value["games"] > 0:
      total.append(key)

  return total


# funcion para contar el número total de partidas por jugar

def count_games_left_total(teams):
  total = 0
  for value in teams.values():
    total += value

  return total // 2



# funcion que calcula la diferencia en audiencia (positiva o negativa) de jugar una partida hipotetica

def count_audience_difference(date, category, scheldule):
  games_audience = category_converter[category]
  new_audience = 0

  for game in scheldule[date]["games"]:
    games_audience += category_converter[game]

  new_audience = scheldule[date]["weight"] * coincidences[len(scheldule[date]["games"]) + 1] * games_audience

  return new_audience - scheldule[date]["audience"], new_audience



# funcion que sirve para crear una partida en el calendario:
# 1. elimina equipos de la lista de equipos que aún no tienen ninguna partida asignado
# 2. guarda una partida en el calendario junto con su audiencia y categoría

def create_game(date, category, new_audience, scheldule, teams):
  reference_scheldule = copy.deepcopy(scheldule)
  reference_teams = copy.deepcopy(teams)

  if category == "AA":
    reference_teams["A"] -= 2
  elif category == "BB":
    reference_teams["B"] -= 2
  elif category == "CC":
    reference_teams["C"] -= 2
  elif category == "AB":
    reference_teams["A"] -= 1
    reference_teams["B"] -= 1
  elif category == "AC":
    reference_teams["A"] -= 1
    reference_teams["C"] -= 1
  elif category == "BC":
    reference_teams["B"] -= 1
    reference_teams["C"] -= 1

  reference_scheldule[date]["games"].append(category)
  reference_scheldule[date]["audience"] = new_audience
  return reference_scheldule, reference_teams



# funcion que transforme el calendario en un array, para poder facilmente crear un DataFrame

def transform_scheldule(data):
  result = []
  for date, values in data.items():
    for game in values["games"]:
      result.append({ 
          "Fecha": date, 
          "Partida": game[0] + " - " + game[1], 
          "Audiencia (en millones)": round(values["weight"] * category_converter[game] * coincidences[len(values["games"])],2) 
          })
  return result

In [9]:
# ALGORITMO POR FUERZA BRUTA

%%time

global_total_audience = 0
global_scheldule = {}

def brute_force(scheldule, teams, total_audience = 0):
  global global_total_audience
  global global_scheldule

  # inicialización de variables
  reference_scheldule = scheldule.copy()
  reference_teams = teams.copy()
  reference_audience = total_audience

  # un bucle para iterar sobre número total de partidas por jugar
  for category in options_avaible(count_available_games(reference_teams)):
    for date in reference_scheldule.keys():
      new_reference_scheldule = reference_scheldule.copy()
      new_reference_audience = reference_audience
      new_reference_teams = reference_teams.copy()

      # calucula de diferencia en audiencia proporcionada por nueva partida
      new_difference, new_audience = count_audience_difference(date, category, reference_scheldule) 
      new_game = { "date": date, "category": category, "new_audience": new_audience }

      # los datos de nueva partida se guardan en el calendario
      new_reference_scheldule, new_reference_teams = create_game(new_game["date"], new_game["category"], new_game["new_audience"], reference_scheldule, reference_teams)
      new_reference_audience += new_difference
      brute_force(new_reference_scheldule, new_reference_teams, new_reference_audience)

  # si el resultado es mejor, sobrescribe el calendario global, cual se devuelve al final del algoritmo
  if reference_audience > global_total_audience:
    global_total_audience = reference_audience
    global_scheldule = reference_scheldule


brute_force(scheldule, teams)

CPU times: user 1min 10s, sys: 7.42 ms, total: 1min 10s
Wall time: 1min 10s


In [10]:
frame = transform_scheldule(global_scheldule)
df = pd.DataFrame(frame)
df = df.append(df.sum(numeric_only=True), ignore_index=True)
df.fillna('',inplace=True)
display(df)
print("La audiencia total del calendario generado es igual a:", '\033[1m' + str(round(global_total_audience,2)) + '\033[0m', "milliones de personas.")

,Fecha,Partida,Audiencia (en millones)
0,S18,B - C,0.60
1,S20,A - A,2.00
2,D16,B - C,0.56
3,D20,A - B,1.30
4,,,4.46


La audiencia total del calendario generado es igual a: 4.46 milliones de personas.


Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

In [11]:
# Es muy dificil calcular la complejidad exacta del algoritmo por fuerza bruta debido a existencia de varios funciones auxiliares y calculo de partidas posibles de jugar.
# Para dar un ejemplo, si tenemos 4 equipos (3xA, 2xB, 1xC) es posible generar partidas de categoría AA, AB, AC, BB, BC, pero dependientemente de cual opción
# elijamos, las siguientes opciones van a diferirse. 

# De todos modos, estimo que la complejidad es de orden factorial.

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

In [12]:
# Para volver a problema original, inicializo otra vez el número de equipos, que anteriormente simplifiqué para testear el algoritmo de fuerza bruta.
teams = {"A": 3, "B": 11, "C":6}

In [13]:
# Mi candidato para dar una solución buena en tiempo limitado es programación dinámica. Es un algoritmo voraz, que significa que no podemos estar seguros
# si el resultado que da será el óptimo, pero seguramente será suficiente para conseguir un número de audiencia alto.

In [14]:
%%time

def dynamic_programming(scheldule, teams):

  # inicialización de variables
  reference_scheldule = copy.deepcopy(scheldule)
  reference_teams = copy.deepcopy(teams)
  total_audience = 0

  # un bucle para iterar sobre número total de partidas por jugar
  for game in range(count_games_left_total(teams)):

    # se calcula un diccionario con categorías de partidas posibles de jugar y se inicializan dos variables de referencia
    games_available = count_available_games(reference_teams)  
    refference_difference = -999
    new_game = {}

    # un bucle para iterar por todas categorías en las que se puede generar una partida
    for category, game_available in games_available.items():
      if game_available["games"] > 0:

        # un bucle para iterar por todas las fechas disponibles
        for date, values in reference_scheldule.items():

          # para cada fecha se calcula la diferencia en audiencia en caso de crear una partida, 
          # teniendo en cuenta audiencia de la fecha, penalización por coincidencia para todas partidas de la fecha y audiencia por categoría de la partida
          # (la diferencia puede tener valor negativo)
          new_difference, new_audience = count_audience_difference(date, category, reference_scheldule) 

          # si la diferencia es mayor que la de referencia, se guarda el nuevo valor de diferencia y detalles de la partida
          if(new_difference > refference_difference):
            refference_difference = new_difference
            new_game = { "date": date, "category": category, "new_audience": new_audience }

    # al final del bucle se crea la partida que atrae el mayor número de aficionados y se guarda la nueva audiencia total
    reference_scheldule, reference_teams = create_game(new_game["date"], new_game["category"], new_game["new_audience"], reference_scheldule, reference_teams)
    total_audience += refference_difference

  return total_audience, reference_scheldule


total_audience, created_scheldule = dynamic_programming(scheldule, teams)

CPU times: user 1.03 ms, sys: 0 ns, total: 1.03 ms
Wall time: 1.04 ms


In [15]:
frame = transform_scheldule(created_scheldule)
df = pd.DataFrame(frame)
df = df.append(df.sum(numeric_only=True), ignore_index=True)
df.fillna('',inplace=True)
display(df)
print("La audiencia total del calendario generado es igual a:", '\033[1m' + str(round(total_audience,2)) + '\033[0m', "milliones de personas.")

,Fecha,Partida,Audiencia (en millones)
0,V20,C - C,0.19
1,S12,B - B,0.50
2,S16,B - B,0.63
3,S18,B - B,0.72
4,S20,A - A,2.00
5,D12,C - C,0.21
6,D16,B - B,0.68
7,D18,B - B,0.45
8,D20,A - B,1.30
9,L20,C - C,0.19


La audiencia total del calendario generado es igual a: 6.86 milliones de personas.


In [16]:
# Otra opción que se podría implementar es un algoritmo de busqueda aleatoria. Aunque en el caso en el que hay 20 equipos de entrada, 
# no ha sido mejor que programación dinámica, puede ser que para un número mayor de entrada sea más eficaz.

In [17]:
def create_random_solution(scheldule, teams):

  # inicialización de variables
  total_audience = 0
  reference_scheldule = scheldule.copy()
  reference_teams = teams.copy()

  # un bucle para iterar sobre número total de partidas por jugar
  for game in range(count_games_left_total(teams)):

    # se calcula un diccionario con categorías de partidas posibles de jugar y se inicializan dos variables de referencia
    games_available = count_available_games(reference_teams)
    new_game = {}

    # de todas posibles categorías de partidas por jugar se elige aleatoriamente una
    games_available_dict = {k: v for k, v in games_available.items() if v["games"] > 0}
    game_category = random.choice(list(games_available_dict.keys()))
    game_values = games_available_dict[game_category]

    # de todas posibles fechas para jugar una partida se elige aleatoriamente una
    dates = reference_scheldule.keys()
    values = reference_scheldule.values()
    date = random.choice(list(dates))

    # se calcula la diferencia en audiencia y se crea la nueva partida en calendario independiemente del resultado
    new_difference, new_audience = count_audience_difference(date, game_category, reference_scheldule) 
    new_game = { "date": date, "category": game_category, "new_audience": new_audience }

    reference_scheldule, reference_teams = create_game(new_game["date"], new_game["category"], new_game["new_audience"], reference_scheldule, reference_teams)
    total_audience += new_difference

  return total_audience, reference_scheldule

In [18]:
%%time

def random_search(scheldule, teams, iterations):
  reference_scheldule = scheldule.copy()
  reference_teams = teams.copy()
  best_total_audience = 0
  best_scheldule = {}

  # generación de calendario en cada iteración
  for i in range(iterations):
    new_total_audience, new_scheldule = create_random_solution(reference_scheldule, reference_teams)

    # si el nuevo calendario es mejor, se guardo el resultado
    if(new_total_audience > best_total_audience and len(new_scheldule["V20"]["games"]) > 0 and len(new_scheldule["L20"]["games"]) > 0 ):
      best_total_audience = new_total_audience
      best_scheldule = new_scheldule

  return best_total_audience, best_scheldule


total_audience, created_scheldule = random_search(scheldule, teams, 100000)

CPU times: user 1min 17s, sys: 10 ms, total: 1min 17s
Wall time: 1min 17s


In [19]:
frame = transform_scheldule(created_scheldule)
df = pd.DataFrame(frame)
df = df.append(df.sum(numeric_only=True), ignore_index=True)
df.fillna('',inplace=True)
display(df)
print("La audiencia total del calendario generado es igual a:", '\033[1m' + str(round(total_audience,2)) + '\033[0m', "milliones de personas.")

,Fecha,Partida,Audiencia (en millones)
0,V20,C - C,0.19
1,S12,B - B,0.50
2,S16,B - B,0.63
3,S18,A - B,1.04
4,S20,B - C,0.56
5,S20,B - B,0.68
6,D12,B - C,0.34
7,D16,B - B,0.68
8,D20,A - A,2.00
9,L20,C - C,0.19


La audiencia total del calendario generado es igual a: 6.79 milliones de personas.


(*)Calcula la complejidad del algoritmo 

Respuesta

In [20]:
# La complejidad del algoritmo de programación dinamica es de orden cuadrático.
# La complejidad del algoritmo de busqueda aleatoria es de orden lineal.

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [21]:
def create_teams(max_value):
  total_number = random.randint(2, max_value)
  total_number = total_number + 1 if total_number % 2 != 0 else total_number

  teams_array = [0,0,0]

  for i in range(total_number):
    teams_array[random.randint(0,2)] += 1

  return {"A": teams_array[0], "B": teams_array[1], "C": teams_array[2]}

teams = create_teams(100)
print(teams)

{'A': 9, 'B': 15, 'C': 14}


Aplica el algoritmo al juego de datos generado

Respuesta

In [22]:
# Para poder generar calendario para número grande de equipos, necesito previamente ampliar el array con penalizaciones por coincidencia
coincidences = [0, 1, 0.75, 0.55, 0.4, 0.3, 0.25, 0.22, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]

In [23]:
# PROGRAMACIÓN DINÁMICA

total_audience, created_scheldule = dynamic_programming(scheldule, teams)
frame = transform_scheldule(created_scheldule)
df = pd.DataFrame(frame)
df = df.append(df.sum(numeric_only=True), ignore_index=True)
df.fillna('',inplace=True)
display(df)
print("La audiencia total del calendario generado es igual a:", '\033[1m' + str(round(total_audience,2)) + '\033[0m', "milliones de personas.")

,Fecha,Partida,Audiencia (en millones)
0,V20,B - B,0.20
1,V20,C - C,0.10
2,V20,C - C,0.10
3,S12,B - B,0.37
4,S12,B - B,0.37
5,S16,A - B,0.68
6,S16,B - B,0.47
7,S18,A - A,1.60
8,S20,A - A,2.00
9,D12,B - B,0.22


La audiencia total del calendario generado es igual a: 10.78 milliones de personas.


In [24]:
# BUSQUEDA ALEATORIA

total_audience, created_scheldule = random_search(scheldule, teams, 100000)
frame = transform_scheldule(created_scheldule)
df = pd.DataFrame(frame)
df = df.append(df.sum(numeric_only=True), ignore_index=True)
df.fillna('',inplace=True)
display(df)
print("La audiencia total del calendario generado es igual a:", '\033[1m' + str(round(total_audience,2)) + '\033[0m', "milliones de personas.")

,Fecha,Partida,Audiencia (en millones)
0,V20,C - C,0.14
1,V20,B - B,0.27
2,S12,B - C,0.23
3,S12,C - C,0.14
4,S12,B - C,0.23
5,S16,A - A,1.05
6,S16,B - B,0.47
7,S18,A - A,1.20
8,S18,B - C,0.45
9,S20,A - A,2.00


La audiencia total del calendario generado es igual a: 10.5 milliones de personas.


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

In [25]:
# No he utilizado ningun otro fuente aparte del manual de la asignatura

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta